### 'narodnaya'으로 시작하는 bigram의 개수를 계산하는 문제

In [1]:
# python3를 기본 python으로 지정
%env PYSPARK_PYTHON=python
%env PYTHONHASHSEED=0

env: PYSPARK_PYTHON=python
env: PYTHONHASHSEED=0


In [2]:
from pyspark import SparkConf, SparkContext
sc = SparkContext(conf=SparkConf().setAppName("MyApp").setMaster("local"))

import re

# article을 단어별로 분리
def parse_article(line):
    try:
        article_id, text = line.rstrip().split('\t', 1)
        text = re.sub("^\W+|\W+$", "", text)
        words = re.split("\W*\s+\W*", text)
        
        return words
    except ValueError as e:
        return []

# bigram을 _로 합친 후에 return
def find_keyword(words):
    try:
        keyword = 'narodnaya'
        res = []
        for index in range(len(words)-1):
            if words[index].lower() == keyword:
                res.append((words[index].lower()+'_'+words[index+1].lower(),1))
        return res
    except ValueError as e:
        return []

In [10]:
# 4 : parition의 개수
file_loc = "/data/wiki/en_articles_part/articles-part"
wiki = sc.textFile(file_loc, 4).map(parse_article).cache()

In [11]:
wiki.getNumPartitions()

4

In [12]:
wiki.count()

4100

In [25]:
wiki.take(1)[0][:10]

['Anarchism',
 'Anarchism',
 'is',
 'often',
 'defined',
 'as',
 'a',
 'political',
 'philosophy',
 'which']

In [20]:
# list로 묶여있는 결과를 flatMap으로 각각 분리
wiki_keywords = wiki.flatMap(find_keyword).filter(lambda x: len(x)>0 )

In [21]:
wiki_keywords.collect()

[('narodnaya_volya', 1),
 ('narodnaya_volya', 1),
 ('narodnaya_volya', 1),
 ('narodnaya_volya', 1),
 ('narodnaya_volya', 1),
 ('narodnaya_volya', 1),
 ('narodnaya_volya', 1),
 ('narodnaya_volya', 1),
 ('narodnaya_volya', 1),
 ('narodnaya_gazeta', 1)]

In [17]:
result = wiki_keywords.reduceByKey(lambda a,b: a+b)

In [18]:
result.collect()

[('narodnaya_volya', 9), ('narodnaya_gazeta', 1)]

In [19]:
result.sortByKey().collect()

[('narodnaya_gazeta', 1), ('narodnaya_volya', 9)]